In [ ]:
%pip install prance requests azure-identity azure-mgmt-resource python-dotenv openapi-spec-validator openai

In [ ]:
import os
import requests
import json
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
from prance import ResolvingParser
from prance.util import resolver
from azure.identity import InteractiveBrowserCredential, TokenCachePersistenceOptions, DefaultAzureCredential
from azure.mgmt.resource import SubscriptionClient

# Load environment variables from .env.local if it exists, otherwise fallback to .env
load_dotenv('.env.local')
load_dotenv()

In [ ]:


# Azure API Management details
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]
resource_group = os.environ["AZURE_RESOURCE_GROUP"]
service_name = os.environ["AZURE_APIM_SERVICE_NAME"]
api_version = os.environ["AZURE_APIM_SERVICE_API_VERSION"]
product_id = os.environ["AZURE_APIM_SERVICE_PRODUCT_ID"]
subscription_key = os.environ["AZURE_APIM_SERVICE_SUBSCRIPTION_KEY"]

#Azure OpenAI details
AZURE_OPENAI_KEY = os.environ["AZURE_OPENAI_KEY"]
AZURE_OPENAI_VERSION = os.environ["AZURE_OPENAI_VERSION"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_MODEL = os.environ["AZURE_OPENAI_MODEL"]


# Authenticate and get an access token
def get_access_token():
    credential = DefaultAzureCredential()
    token = credential.get_token("https://management.azure.com/.default")
    return token.token

# Fetch all APIs from Azure API Management
def fetch_apis(access_token):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    url = f'https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.ApiManagement/service/{service_name}/apis?api-version={api_version}'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()['value']

# Function to fetch APIs that belong to a certain product
def fetch_apis_by_product(access_token, product_id):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    url = f'https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.ApiManagement/service/{service_name}/products/{product_id}/apis?api-version={api_version}'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()['value']

# Get the OpenAPI JSON file from the export endpoint
def fetch_openapi_spec(api_id, access_token):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    url = f'https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.ApiManagement/service/{service_name}/apis/{api_id}?export=true&format=openapi&api-version={api_version}'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    #return response.json()
    return response.text

# Parse the OpenAPI spec and create an array of OpenAI tool calls
def parse_openapi_spec(openapi_spec):
    parser = ResolvingParser(spec_string=openapi_spec, resolve_types = resolver.RESOLVE_FILES, strict=False)
    parsed_spec = parser.specification
    tool_calls = []
    function_api_map = {}

     # Extract the server URL from the OpenAPI spec
    servers = parsed_spec.get('servers', [])
    if servers and 'url' in servers[0]:
        base_url = servers[0]['url']
    else:
        base_url = ''  # Handle cases where server URL is missing

    for path, methods in parsed_spec['paths'].items():
        for method, details in methods.items():
            function_name = details.get('operationId', '')
            parameters = details.get('parameters', [])

            param_properties = {}
            required_params = []

            # Handle parameters in 'parameters' field
            parameters = details.get('parameters', [])
            for param in parameters:
                param_name = param['name']
                param_in = param.get('in', 'query')
                param_schema = param.get('schema', {})
                param_type = param_schema.get('type', 'string')
                param_description = param.get('description', '')
                param_required = param.get('required', False)

                # Build parameter properties
                param_properties[param_name] = {
                    "type": param_type,
                    "description": param_description
                }
                if param_required:
                    required_params.append(param_name)

            # Handle parameters in 'requestBody' field
            request_body = details.get('requestBody')
            if request_body:
                content = request_body.get('content', {})
                # Assuming 'application/json' content type
                json_schema = content.get('application/json', {}).get('schema', {})
                if json_schema:
                    body_properties = json_schema.get('properties', {})
                    body_required = json_schema.get('required', [])
                    for prop_name, prop_schema in body_properties.items():
                        prop_type = prop_schema.get('type', 'string')
                        prop_description = prop_schema.get('description', '')
                        # Add to parameter properties
                        param_properties[prop_name] = {
                            "type": prop_type,
                            "description": prop_description
                        }
                        if prop_name in body_required:
                            required_params.append(prop_name)
            print(f"    - Creating Function for: {function_name}")

            # Note: Function name has a 

            tool_call = {
                "type": "function",
                "function": {
                    "name": function_name,
                    "description": details.get('summary', ''),
                    "parameters": {
                        "type": "object",
                        "properties": param_properties,
                        "required": required_params
                    }
                }
            }
            tool_calls.append(tool_call)
            # Add to function_api_map
            function_api_map[function_name] = {
                'method': method.upper(),
                'path': path,
                'parameters': parameters,
                'request_body': request_body,
                'base_url': base_url
            }

    return tool_calls, function_api_map

# Define execute_function_call to make API requests
def execute_function_call(function_name, arguments, function_api_map):
    api_details = function_api_map.get(function_name)
    if not api_details:
        return f"Function {function_name} is not available."

    method = api_details['method']
    path = api_details['path']
    params = api_details['parameters']
    request_body = api_details.get('request_body', {})
    base_url = api_details.get('base_url', '')
    args = json.loads(arguments)

    # Construct the full URL using base_url and path
    if base_url.endswith('/') and path.startswith('/'):
        url = f'{base_url[:-1]}{path}'
    elif not base_url.endswith('/') and not path.startswith('/'):
        url = f'{base_url}/{path}'
    else:
        url = f'{base_url}{path}'

    query_params = {}
    header_params = {}
    path_params = {}
    # Initialize the body payload
    body_payload = None

    # Handle parameters from 'parameters' field
    for param in params:
        name = param['name']
        location = param.get('in', 'query')
        value = args.get(name)

        if location == 'path':
            url = url.replace(f'{{{name}}}', str(value))
        elif location == 'query':
            query_params[name] = value
        elif location == 'header':
            header_params[name] = value

    # Handle request body parameters
    if request_body:
        content = request_body.get('content', {})
        json_schema = content.get('application/json', {}).get('schema', {})
        if json_schema:
            body_payload = {}
            body_properties = json_schema.get('properties', {})
            for prop_name in body_properties.keys():
                if prop_name in args:
                    body_payload[prop_name] = args[prop_name]

    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,  # Ensure this is defined
        'Content-Type': 'application/json'
    }
    headers.update(header_params)

    # Make the API request
    if method == 'GET':
        response = requests.get(url, headers=headers, params=query_params)
    else:
        response = requests.request(
            method,
            url,
            headers=headers,
            params=query_params,
            json=body_payload
        )

    response.raise_for_status()
    return response.text

# Function to call Azure OpenAI with the generated tool calls
def call_azure_openai(prompt, query, tool_calls, function_api_map):
    # Set up the OpenAI API credentials and endpoint for Azure OpenAI
    client = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_VERSION
    )

    # Prepare the messages and functions
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": query}
    ]

    # Make the API call to Azure OpenAI
    while True:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_MODEL,
            messages=messages,
            tools=tool_calls,
            tool_choice="auto",
        )

        # Process the model's response
        response_message = response.choices[0].message
        messages.append(response_message)

        #print("Model's response:")  
        #print(response_message) 

        print("Response content:")
        print(response_message.content)

        if response_message.tool_calls:
            for tool_call in response_message.tool_calls:

                function_name = tool_call.function.name
                function_args = tool_call.function.arguments

                print(f"    - Function call: {function_name} with arguments: {function_args}")

                # Execute the function call
                function_response = execute_function_call(function_name, function_args, function_api_map)

                # Add the function response to messages
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response
                })
        else:
            # No more function calls; print the assistant's final response
            print("Final response:")
            print(response_message.content)
            break

# Main function to fetch and parse all OpenAPI specs
def main():
    access_token = get_access_token()
    # Fetch All API's
    #apis = fetch_apis(access_token)
    # Or fetch APIs that belong to a certain product
    apis = fetch_apis_by_product(access_token, product_id)
    all_tool_calls = []
    function_api_map = {}

    for api in apis:
        api_id = api['name']
        openapi_spec = fetch_openapi_spec(api_id, access_token)
        tool_calls, api_map = parse_openapi_spec(openapi_spec)
        all_tool_calls.extend(tool_calls)
        function_api_map.update(api_map)

    # Print or return the array of OpenAI tool calls
    #print(json.dumps(all_tool_calls, indent=2))

     # Call Azure OpenAI with the generated tool calls
    prompt = (
                "Instructions:\n"
                " - Break the task into steps, and output the result of each step as you perform it.\n"
                " - You are an AI assistant that helps with calling APIs to generate useful information based on a user's question.\n"
                " - Use the proper function calls to get information that will be useful to the user.\n"
                " - If one function call depends on the output of another, make sure to call them in order and use the outputs appropriately.\n"
                " - Include what you are thinking, working on, and next steps in your responses, and ask for more information if needed.\n"
                " - If you don't know something, and are not able to ask the user for more information, or can't call an API, you can say 'I don't know'.\n"
                " - Always format an email as HTML. Ensure the content is well orangized and use bullet lists or tables where necessary.\n"
            )
    query = "What is the weather in Orlando, FL? What product's would work for me with the current temperature? Once you have it, send an email to adam.hockemeyer@microsoft.com with the details."
    #query = "What products are available that support 70 degrees temperature?"
    call_azure_openai(prompt, query, all_tool_calls, function_api_map)

if __name__ == '__main__':
    main()